# **1. Perkenalan Dataset**


Dataset: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system

Dataset Rekomendasi Film yang diunggah oleh Parasharmanas di Kaggle berisi informasi tentang berbagai film serta rating yang diberikan oleh pengguna. Dataset ini biasanya digunakan untuk membangun sistem rekomendasi berbasis konten yang berfokus pada genre, deskripsi, dan fitur lainnya untuk memberikan rekomendasi film kepada pengguna.


Dataset ini terdiri dari dua file CSV utama:

- movies.csv: Berisi informasi tentang film, termasuk:

        movieId: ID unik untuk setiap film.

        title: Nama film.

        genres: Genre film, yang dapat terdiri dari beberapa kategori, misalnya "Action", "Comedy", "Drama", dll.

- ratings.csv: Berisi rating yang diberikan oleh pengguna terhadap film, dengan kolom-kolom berikut:

        userId: ID unik untuk setiap pengguna.

        movieId: ID film yang diberikan rating.

        rating: Rating yang diberikan oleh pengguna pada skala 1 hingga 5.

        timestamp: Waktu saat rating diberikan.




# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install -q kaggle
from google.colab import files
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dot, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import random
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding, Dense, Dot, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Concatenate, Input
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

# **3. Memuat Dataset**

Untuk memuat dataset Anda menggunakan pustaka pandas sesuai dengan dataset yang telah Anda unduh dari Kaggle, Anda hanya perlu memeriksa beberapa baris pertama untuk memastikan bahwa data telah dimuat dengan benar. Anda sudah memuat dataset menggunakan pd.read_csv() di tahap sebelumnya, jadi Anda hanya perlu memeriksa beberapa baris pertama dari data.

Mengunggah file ke dalam Notebook colab. Pada Proyek ini menggunakan file Kaggle.json untuk otentikasi dataset yang bersumber dari kaggle

In [2]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


mengonfigurasi Kaggle API untuk mengakses dataset dari Kaggle menggunakan file kredensial yang telah diunggah.

In [3]:
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
os.rename('kaggle.json', os.path.expanduser('~/.kaggle/kaggle.json'))
!chmod 600 ~/.kaggle/kaggle.json


mengunduh file ZIP yang berisi dataset Movie Recommendation System ke direktori.

In [4]:
!kaggle datasets download -d parasharmanas/movie-recommendation-system

Dataset URL: https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system
License(s): ODbL-1.0
 76% 126M/165M [00:00<00:00, 379MB/s] 
100% 165M/165M [00:00<00:00, 381MB/s]


mengekstrak file ZIP yang telah diunduh dari Kaggle ke direktori tertentu di Google Colab

In [5]:
!unzip -q movie-recommendation-system.zip -d /content/movie-recommendation-system

 menampilkan daftar file yang ada di dalam folder /content/movie-recommendation-system, untuk memverifikasi file apa saja yang telah berhasil diekstrak dari file ZIP.

In [6]:
extracted_dir = "/content/movie-recommendation-system"
print("Isi direktori dataset:")
print(os.listdir(extracted_dir))

Isi direktori dataset:
['movies.csv', 'ratings.csv']


 menyimpan lokasi file dalam dua variabel berbeda, yaitu movies_path untuk file film dan ratings_path untuk file rating dan kedua variabel ini akan digunakan untuk memuat data ke dalam DataFrame.

 insight: Menetapkan path untuk file CSV yang berisi data film dan data rating.

In [7]:
movies_path = '/content/movie-recommendation-system/movies.csv'
ratings_path = '/content/movie-recommendation-system/ratings.csv'

- movies_df berisi data tentang film
- ratings_df berisi data tentang rating yang diberikan pengguna kepada film-film tertentu


insight: Memuat file CSV untuk data film dan rating ke dalam dua DataFrame terpisah

In [8]:
movies_df = pd.read_csv(movies_path)
ratings_df = pd.read_csv(ratings_path)

Mengambil hanya 10.000 baris pertama dari data

insight: Dilakukan untuk mengurangi ukuran data agar lebih mudah diproses.

In [9]:
movies_slice = movies_df.head(10000)
ratings_slice = ratings_df.head(10000)

Menyimpan hasilnya ke file CSV

insight: Menyimpan data yang telah dipotong ke dalam csv

In [10]:
movies_slice.to_csv("movies_slice.csv", index=False)
ratings_slice.to_csv("ratings_slice.csv", index=False)

dataset yang sudah dislicing

insight: Memuat kembali file CSV yang telah disimpan memastikan bahwa proses pemotongan dan penyimpanan berjalan dengan baik dan data siap untuk digunakan lebih lanjut.

In [11]:
movies_df = pd.read_csv("movies_slice.csv")
ratings_df = pd.read_csv("ratings_slice.csv")

# **4. Data Understanding**

Menampilkan jumlah data pada masing-masing dataset

insight: Hasil yang ditampilkan adalah jumlah total data untuk film dan rating, serta jumlah total keseluruhan dari kedua dataset tersebut.

In [12]:
print(f"Jumlah data movie: {len(movies_df)}")
print(f"Jumlah data rating: {len(ratings_df)}")
print(f"Total data: {len(movies_df) + len(ratings_df)}")

Jumlah data movie: 10000
Jumlah data rating: 10000
Total data: 20000


- Menampilkan contoh data movie
- Menampilkan contoh data rating

insight: menampilkan 5 baris pertama dari data film dan rating

In [13]:
print("\nContoh data movie:")
print(movies_df.head())
print("\nContoh data rating:")
print(ratings_df.head())


Contoh data movie:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Contoh data rating:
   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510


- Menampilkan info struktur data movie
- Menampilkan info struktur data rating

insight: Dataset movies_df terdiri dari 10.000 film, dengan kolom movieId, title, dan genres. Kolom title dan genres berisi tipe data objek (teks), sementara movieId adalah angka. Dataset ratings_df terdiri dari 10.000 rating dengan kolom userId, movieId, rating, dan timestamp, yang semuanya bertipe numerik.

In [14]:
print("\nInfo struktur data movie:")
print(movies_df.info())
print("\nInfo struktur data rating:")
print(ratings_df.info())


Info struktur data movie:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10000 non-null  int64 
 1   title    10000 non-null  object
 2   genres   10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB
None

Info struktur data rating:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     10000 non-null  int64  
 1   movieId    10000 non-null  int64  
 2   rating     10000 non-null  float64
 3   timestamp  10000 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 312.6 KB
None


- Menampilkan statistik deskriptif data movie
- Menampilkan statistik deskriptif data rating

insight: Menampilkan statistik deskriptif yang memberikan informasi tentang jumlah data film dan statistik untuk kolom movieId. Kolom title tidak memiliki statistik numerik karena berisi teks. Menampilkan statistik deskriptif untuk kolom rating, memberikan gambaran tentang distribusi nilai rating, serta nilai minimum, maksimum, dan rata-rata rating.

In [15]:
print("\nStatistik deskriptif data movie:")
print(movies_df.describe(include='all'))
print("\nStatistik deskriptif data rating:")
print(ratings_df.describe(include='all'))


Statistik deskriptif data movie:
             movieId                 title genres
count   10000.000000                 10000  10000
unique           NaN                  9999    780
top              NaN  Men with Guns (1997)  Drama
freq             NaN                     2   1703
mean     8410.744200                   NaN    NaN
std      9461.921046                   NaN    NaN
min         1.000000                   NaN    NaN
25%      2590.750000                   NaN    NaN
50%      5105.500000                   NaN    NaN
75%      7873.500000                   NaN    NaN
max     33779.000000                   NaN    NaN

Statistik deskriptif data rating:
             userId        movieId        rating     timestamp
count  10000.000000   10000.000000  10000.000000  1.000000e+04
mean      35.504400   16481.889300      3.599400  1.179452e+09
std       23.782269   31968.753804      1.005321  2.113199e+08
min        1.000000       1.000000      0.500000  8.307862e+08
25%       13.000

# **5. Univariate Exploratory Data Analysis**

Eksplorasi Judul Film sering muncul

insight: Dataset memiliki 2 judul film unik. Film dengan judul paling sering muncul adalah "Men with Guns (1997)" yang muncul 2 kali, sementara judul lainnya hanya muncul sekali. Hasil ini menunjukkan bahwa ada duplikasi atau pengulangan pada beberapa film dalam dataset.

In [16]:
print("\n--- Eksplorasi variabel 'Title' ---")
title_counts = movies_df['title'].value_counts()
print(f"Jumlah unik judul film: {title_counts.nunique()}")
print(f"Contoh 5 judul film yang paling sering muncul: {title_counts.head(5)}")


--- Eksplorasi variabel 'Title' ---
Jumlah unik judul film: 2
Contoh 5 judul film yang paling sering muncul: title
Men with Guns (1997)                               2
Rules of the Game, The (La règle du jeu) (1939)    1
Song Remains the Same, The (1976)                  1
Seven Brides for Seven Brothers (1954)             1
Kiss of the Spider Woman (1985)                    1
Name: count, dtype: int64


Eksplorasi Variabel Genre Film sering muncul

insight: Terdapat 19 genre film unik dalam dataset. Genre yang paling sering muncul adalah "Drama" dengan 5057 kemunculan, diikuti oleh genre lainnya seperti "Comedy", "Romance", dan "Thriller". Ini memberikan gambaran bahwa genre "Drama" lebih dominan dalam dataset dibandingkan genre lainnya.

In [17]:
print("\n Eksplorasi variabel 'Genre'")
genre_counts = movies_df['genres'].str.split('|', expand=True).stack().value_counts()
print(f"Jumlah unik genre film: {genre_counts.nunique()}")
print(f"5 genre film yang paling banyak muncul: {genre_counts.head(5)}")


 Eksplorasi variabel 'Genre'
Jumlah unik genre film: 19
5 genre film yang paling banyak muncul: Drama       5057
Comedy      3464
Romance     1689
Thriller    1579
Action      1358
Name: count, dtype: int64


Eksplorasi Variabel Rating dari user

insight: Statistik deskriptif untuk rating menunjukkan bahwa dataset memiliki 10.000 rating dengan nilai rata-rata 3.59 dan standar deviasi 1.06. Nilai rating terendah adalah 0.5, sementara yang tertinggi adalah 5.0. Terdapat 10 nilai unik dalam rating yang digunakan, yang menunjukkan bahwa rating dibatasi pada angka-angka tertentu (mungkin 0.5 hingga 5.0).

In [18]:
print("\n Eksplorasi variabel 'Rating'")
rating_stats = ratings_df['rating'].describe()
print("Statistik deskriptif dari rating:")
print(rating_stats)
print(f"Jumlah nilai unik pada rating: {ratings_df['rating'].nunique()}")
print(f"Rating terendah: {ratings_df['rating'].min()}")
print(f"Rating tertinggi: {ratings_df['rating'].max()}")


 Eksplorasi variabel 'Rating'
Statistik deskriptif dari rating:
count    10000.000000
mean         3.599400
std          1.005321
min          0.500000
25%          3.000000
50%          4.000000
75%          4.000000
max          5.000000
Name: rating, dtype: float64
Jumlah nilai unik pada rating: 10
Rating terendah: 0.5
Rating tertinggi: 5.0


Eksplorasi distribusi rating per film

insight: Dataset ini terdiri dari 3.287 film yang memiliki rating. Rata-rata rating per film adalah 3.41, dengan standar deviasi 0.88. Rating per film bervariasi, dengan rating terendah adalah 0.5 dan tertinggi adalah 5.0. Statistik ini memberikan gambaran bahwa meskipun ada banyak film dengan rating, distribusi rating per film menunjukkan adanya variasi dalam penerimaan film oleh pengguna.

In [19]:
print("\n--- Eksplorasi rating per film ---")
average_rating_per_movie = ratings_df.groupby('movieId')['rating'].mean()
print(f"Jumlah film yang memiliki rating: {average_rating_per_movie.count()}")
print(f"Rata-rata rating per film:\n{average_rating_per_movie.describe()}")


--- Eksplorasi rating per film ---
Jumlah film yang memiliki rating: 3287
Rata-rata rating per film:
count    3287.000000
mean        3.418694
std         0.877573
min         0.500000
25%         3.000000
50%         3.500000
75%         4.000000
max         5.000000
Name: rating, dtype: float64


Eksplorasi rating berdasarkan user

insight:  distribusi rating per pengguna menunjukkan adanya beberapa pengguna yang sangat aktif dalam memberikan rating, sementara yang lainnya hanya memberikan beberapa rating.

In [20]:
print("\n Eksplorasi rating per pengguna")
ratings_count_per_user = ratings_df.groupby('userId')['rating'].count()
print(f"Jumlah pengguna yang memberikan rating: {ratings_count_per_user.count()}")
print(f"Rata-rata rating per pengguna:\n{ratings_count_per_user.describe()}")


 Eksplorasi rating per pengguna
Jumlah pengguna yang memberikan rating: 75
Rata-rata rating per pengguna:
count     75.000000
mean     133.333333
std      167.914822
min       20.000000
25%       31.500000
50%       60.000000
75%      169.000000
max      813.000000
Name: rating, dtype: float64


# **6. Data Preprocessing**

Menggabungkan Data Movies dan Ratings

insight: Dataset ratings_df dan movies_df digabungkan berdasarkan kolom movieId menggunakan metode left join, sehingga informasi genre dari dataset film ditambahkan ke dataset rating.

In [21]:
movie_ratings_df = pd.merge(ratings_df, movies_df[['movieId', 'genres']], on='movieId', how='left')

Menampilkan baris awal setelah penggabungan data

insight: Setelah penggabungan, data yang ditampilkan meliputi userId, movieId, rating, timestamp, dan genres yang berasal dari kedua dataset tersebut.

In [22]:
print("\n Cek data Setelah Penggabungan:")
print(movie_ratings_df.head())


 Cek data Setelah Penggabungan:
   userId  movieId  rating   timestamp                       genres
0       1      296     5.0  1147880044  Comedy|Crime|Drama|Thriller
1       1      306     3.5  1147868817                        Drama
2       1      307     5.0  1147868828                        Drama
3       1      665     5.0  1147878820             Comedy|Drama|War
4       1      899     3.5  1147868510       Comedy|Musical|Romance


Statistik Deskriptif setelah penggabungan data

insigth: Menampilkan statistik deskriptif untuk seluruh data setelah penggabungan. Kolom yang terlihat meliputi userId, movieId, rating, timestamp, dan genres. Statistik mencakup jumlah total data (count), nilai rata-rata, standar deviasi, nilai minimum dan maksimum, serta beberapa informasi lainnya.
Kolom genres menunjukkan adanya variasi genre yang cukup banyak dengan frekuensi genre tertentu yang lebih sering muncul.

In [23]:
print("\nStatistik Deskriptif Data:")
print(movie_ratings_df.describe(include='all'))


Statistik Deskriptif Data:
              userId        movieId        rating     timestamp genres
count   10000.000000   10000.000000  10000.000000  1.000000e+04   8194
unique           NaN            NaN           NaN           NaN    473
top              NaN            NaN           NaN           NaN  Drama
freq             NaN            NaN           NaN           NaN    597
mean       35.504400   16481.889300      3.599400  1.179452e+09    NaN
std        23.782269   31968.753804      1.005321  2.113199e+08    NaN
min         1.000000       1.000000      0.500000  8.307862e+08    NaN
25%        13.000000    1073.000000      3.000000  9.806502e+08    NaN
50%        33.000000    2571.000000      4.000000  1.167743e+09    NaN
75%        59.000000    6934.500000      4.000000  1.349690e+09    NaN
max        75.000000  203519.000000      5.000000  1.574195e+09    NaN


# **7. Data Preparation**

Cek jumlah missing values

insight: memiliki 1806 missing values dan menghapus nilai yang hilang dari dataset menggunakan dropna() dan menyimpannya di movie_ratings_df_cleaned.

In [24]:
print(f"Jumlah missing values sebelum diisi:\n{movie_ratings_df.isnull().sum()}")
movie_ratings_df_cleaned = movie_ratings_df.dropna()

Jumlah missing values sebelum diisi:
userId          0
movieId         0
rating          0
timestamp       0
genres       1806
dtype: int64


Cek kembali missing values

insight: Setelah menghapus nilai yang hilang, tidak ada lagi nilai hilang pada dataset yang sudah dibersihkan (movie_ratings_df_cleaned).

In [25]:
print(f"\nJumlah missing values setelah dihapus:\n{movie_ratings_df_cleaned.isnull().sum()}")


Jumlah missing values setelah dihapus:
userId       0
movieId      0
rating       0
timestamp    0
genres       0
dtype: int64


Menghapus duplikasi

insight: Sebelum menghapus duplikat, dataset movie_ratings_df memiliki 10.000 baris. Proses ini menggunakan drop_duplicates() untuk menghapus duplikat dari dataset dan menyimpannya di movie_ratings_df_cleaned.

In [26]:
print(f"\nJumlah baris sebelum menghapus duplikat: {len(movie_ratings_df)}")
movie_ratings_df_cleaned = movie_ratings_df.drop_duplicates()


Jumlah baris sebelum menghapus duplikat: 10000


Cek apakah ada duplikasi setelah dihapus

insight: Setelah menghapus duplikat, jumlah baris pada dataset yang sudah dibersihkan tetap 10.000, yang menunjukkan bahwa tidak ada duplikat yang tersisa.

In [27]:
print(f"\nJumlah baris setelah menghapus duplikat: {len(movie_ratings_df_cleaned)}")


Jumlah baris setelah menghapus duplikat: 10000


Tipe data setiap kolomnya

insight: Ini memastikan bahwa kolom-kolom yang berisi data numerik (seperti userId, movieId, dan rating) memiliki tipe data yang sesuai.

In [28]:
print("\nTipe data dari setiap kolom:")
print(movie_ratings_df.dtypes)



Tipe data dari setiap kolom:
userId         int64
movieId        int64
rating       float64
timestamp      int64
genres        object
dtype: object


Pastikan kolom 'userId', 'movieId', dan 'rating' adalah tipe numerik

insight: Tipe data untuk kolom userId, movieId, dan rating dipastikan sesuai dengan tipe numerik yang tepat menggunakan metode .astype(int) untuk userId dan movieId, serta .astype(float) untuk rating.

In [29]:
movie_ratings_df['userId'] = movie_ratings_df['userId'].astype(int)
movie_ratings_df['movieId'] = movie_ratings_df['movieId'].astype(int)
movie_ratings_df['rating'] = movie_ratings_df['rating'].astype(float)

- Cek hasil pembersihan data
- Statistik deskriptif untuk memahami distribusi data


insight: Setelah memastikan tipe data yang benar, data diperiksa dan statistik deskriptif dihitung untuk memahami distribusi data. Data yang ditampilkan mencakup kolom userId, movieId, rating, timestamp, dan genres dengan statistik deskriptif yang mencakup jumlah data, rata-rata, standar deviasi, nilai minimum dan maksimum, serta kuartil pertama dan ketiga.

In [30]:
print("\nContoh data setelah pembersihan:")
print(movie_ratings_df.head())
print("\nStatistik Deskriptif Data Setelah Pembersihan:")
print(movie_ratings_df.describe(include='all'))


Contoh data setelah pembersihan:
   userId  movieId  rating   timestamp                       genres
0       1      296     5.0  1147880044  Comedy|Crime|Drama|Thriller
1       1      306     3.5  1147868817                        Drama
2       1      307     5.0  1147868828                        Drama
3       1      665     5.0  1147878820             Comedy|Drama|War
4       1      899     3.5  1147868510       Comedy|Musical|Romance

Statistik Deskriptif Data Setelah Pembersihan:
              userId        movieId        rating     timestamp genres
count   10000.000000   10000.000000  10000.000000  1.000000e+04   8194
unique           NaN            NaN           NaN           NaN    473
top              NaN            NaN           NaN           NaN  Drama
freq             NaN            NaN           NaN           NaN    597
mean       35.504400   16481.889300      3.599400  1.179452e+09    NaN
std        23.782269   31968.753804      1.005321  2.113199e+08    NaN
min         1

indeks berdasarkan judul film

insight: Secara keseluruhan, langkah ini memastikan bahwa data film diindeks berdasarkan judulnya, dengan duplikat yang dihapus untuk memastikan keunikan data berdasarkan judul film.

In [31]:
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()

## **Modeling**


Menggunakan TfidfVectorizer untuk menghitung representasi genre

insight:  proses ini bertujuan untuk mengonversi genre teks dari dataset film menjadi representasi numerik yang dapat digunakan untuk analisis lebih lanjut atau dalam model pembelajaran mesin.

In [32]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])


Menghitung cosine similarity antar film

insight:  sebuah matriks similiaritas yang menunjukkan seberapa mirip setiap pasangan film berdasarkan genre mereka.

In [33]:
cosine_sim = cosine_similarity(tfidf_matrix)


Membuat DataFrame untuk cosine similarity

insight: DataFrame ini menyajikan nilai cosine similarity antara setiap pasangan film dalam bentuk tabel yang lebih mudah dibaca. Contoh data yang acak (5 film pertama) ditampilkan menggunakan sample(5) untuk memberikan gambaran tentang bagaimana film-film saling berhubungan berdasarkan genre mereka. Proses ini menghitung tingkat kesamaan antara film berdasarkan genre mereka, dan hasilnya disajikan dalam bentuk tabel cosine similarity yang memungkinkan untuk melihat film mana yang memiliki genre yang paling mirip dengan film lainnya.

In [34]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=movies_df['title'], columns=movies_df['title'])


In [35]:
cosine_sim_df = cosine_sim_df * 100
cosine_sim_df = cosine_sim_df.round(0).astype(int)
print('Shape:', cosine_sim_df.shape)
print("\nContoh cosine similarity (5 film acak):")
print(cosine_sim_df.sample(5, axis=1).sample(10, axis=0))

Shape: (10000, 10000)

Contoh cosine similarity (5 film acak):
title                                               Sphinx (1981)  \
title                                                               
Horror Planet (a.k.a. Inseminoid) (1981)                        0   
Sleepless in Seattle (1993)                                     0   
8 Seconds (1994)                                                0   
Kiss of the Spider Woman (1985)                                 0   
Bangkok Dangerous (2000)                                        0   
Africa Screams (1949)                                           0   
Return of the Street Fighter (Satsujin ken 2) (...              0   
Fade to Black (2004)                                            0   
Tanguy (2001)                                                   0   
Avanti! (1972)                                                  0   

title                                               Mother Night (1996)  \
title                            

Sistem Rekomendasi

insight: Rekomendasi film ditampilkan bersama dengan genre mereka.

In [36]:
def recommend(title, cosine_sim=cosine_sim, df=movies_df):
    if title not in indices:
        print(f"Judul '{title}' tidak ditemukan.")
        return

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
    movie_indices = [i[0] for i in sim_scores]
    recommended_movies = df.iloc[movie_indices][['title', 'genres']].copy()
    recommended_movies.reset_index(drop=True, inplace=True)
    recommended_movies.index += 1

    print(f"\nRekomendasi film mirip dengan: '{title}':\n")
    return recommended_movies

Rekomendasi Film

insight: Hasilnya menampilkan 10 film yang memiliki kemiripan genre tertinggi. Fungsi ini memberikan rekomendasi film berdasarkan genre yang mirip dengan film yang dicari, menggunakan cosine similarity untuk mengukur kemiripan antar film. Output yang diberikan adalah daftar film yang mirip beserta genre mereka.

In [37]:
recommendations = recommend("Snow Day (2000)")
print(recommendations)


Rekomendasi film mirip dengan: 'Snow Day (2000)':

                                                title  genres
1                                   Four Rooms (1995)  Comedy
2               Ace Ventura: When Nature Calls (1995)  Comedy
3                                     Bio-Dome (1996)  Comedy
4                                       Friday (1995)  Comedy
5                                  Black Sheep (1996)  Comedy
6                                    Mr. Wrong (1996)  Comedy
7                                Happy Gilmore (1996)  Comedy
8   Happiness Is in the Field (Bonheur est dans le...  Comedy
9                      Steal Big, Steal Little (1995)  Comedy
10                      Flirting With Disaster (1996)  Comedy


# **9. Evaluasi**


Nilai Precision nya

insight: Fungsi ini menghitung dan menampilkan nilai precision dari rekomendasi film berdasarkan threshold cosine similarity. Hasil precision 1.00 berarti bahwa semua rekomendasi film untuk "Snow Day (2000)" relevan.

In [38]:
def calculate_precision_for_recommendation(title, cosine_sim=cosine_sim, df=movies_df, threshold=0.5):
    if title not in indices:
        print(f"Judul '{title}' tidak ditemukan.")
        return 0

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]

    relevant_recommendations = sum([1 for score in sim_scores if score[1] > threshold])
    total_recommendations = len(sim_scores)
    precision = relevant_recommendations / total_recommendations if total_recommendations > 0 else 0

    return precision

title = "Snow Day (2000)"
precision = calculate_precision_for_recommendation(title, cosine_sim=cosine_sim, df=movies_df, threshold=0.5)

print(f"Precision untuk rekomendasi film mirip dengan '{title}': {precision:.2f}")

Precision untuk rekomendasi film mirip dengan 'Snow Day (2000)': 1.00
